# 📝 Lab #3-4 정답지: 통신사 고객 이탈 예측 모델 해석

### 1. 데이터 준비 및 모델 훈련

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# 데이터 로드
path = '../../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)

# --- 데이터 전처리 ---
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)
df.drop('customerID', axis=1, inplace=True)
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop('Churn', axis=1)
y = df['Churn']
feature_names = X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 훈련
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_train, y_train)

# 모델 성능 평가
preds = lgbm.predict(X_test)
proba = lgbm.predict_proba(X_test)[:, 1]
print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, proba):.4f}")

Accuracy: 0.8038
ROC AUC: 0.8437


### 2. 전역적 해석 (Global Interpretation)

#### [문제 1] Permutation Importance 계산 (정답)

In [2]:
from sklearn.inspection import permutation_importance

# ROC AUC를 점수 기준으로 Permutation Importance 계산
result = permutation_importance(
    lgbm, X_test, y_test, scoring='roc_auc', n_repeats=10, random_state=42, n_jobs=-1
)

# 결과를 DataFrame으로 정리
importances = pd.DataFrame(
    {'feature': feature_names,
     'importance_mean': result.importances_mean,
     'importance_std': result.importances_std}
).sort_values('importance_mean', ascending=False)

# 상위 10개 특성 시각화
fig = px.bar(importances.head(10).sort_values('importance_mean'),
             x='importance_mean',
             y='feature',
             error_x='importance_std',
             title='Permutation Importance (Top 10 Features)',
             labels={'importance_mean': 'Importance (ROC AUC Drop)', 'feature': 'Feature'},
             orientation='h')

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

#### [문제 2] SHAP Summary Plot (정답)

In [3]:
import shap
shap.initjs()

# TreeExplainer를 사용하여 SHAP 값 계산
explainer = shap.TreeExplainer(lgbm)
shap_values = explainer.shap_values(X_test)

# Summary Plot 시각화
# shap_values[1]은 'Churn=1' 클래스에 대한 기여도를 의미합니다.
shap.summary_plot(shap_values[1], X_test, plot_type="dot")

### 3. 지역적 해석 (Local Interpretation)

#### [문제 3] 특정 고객 예측 해석 (SHAP Force Plot) (정답)

In [4]:
# 이탈(Churn=1)로 올바르게 예측한 고객 인덱스 찾기
churn_correct_indices = np.where((y_test.values == 1) & (preds == 1))[0]
churn_customer_idx = churn_correct_indices[0] # 첫 번째 고객 선택
print(f"이탈(Churn=1)로 올바르게 예측된 고객 인덱스: {churn_customer_idx}")

# 해당 고객에 대한 Force Plot 시각화
# shap_values[1]은 'Churn=1' 클래스에 대한 값
shap.force_plot(explainer.expected_value[1], 
                shap_values[1][churn_customer_idx], 
                X_test.iloc[churn_customer_idx])

이탈(Churn=1)로 올바르게 예측된 고객 인덱스: 1


#### [문제 4] 특정 고객 예측 해석 (LIME) (정답)

In [5]:
import lime
import lime.lime_tabular

# LIME Explainer 객체 생성
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=feature_names,
    class_names=['Not Churn', 'Churn'],
    mode='classification'
)

# 비이탈(Churn=0)로 올바르게 예측한 고객 인덱스 찾기
no_churn_correct_indices = np.where((y_test.values == 0) & (preds == 0))[0]
no_churn_customer_idx = no_churn_correct_indices[0]
print(f"비이탈(Churn=0)로 올바르게 예측된 고객 인덱스: {no_churn_customer_idx}")

# 해당 고객에 대한 LIME 설명 생성
explanation = lime_explainer.explain_instance(
    X_test.iloc[no_churn_customer_idx].values,
    lgbm.predict_proba,
    num_features=6
)

# 결과 시각화
explanation.show_in_notebook(show_table=True)

비이탈(Churn=0)로 올바르게 예측된 고객 인덱스: 0


<iframe src="data:text/html;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPgo8bWV0YSBodHRwLWVxdWl2PSJDb250ZW50LVR5cGUiIGNvbnRlbnQ9InRleHQvaHRtbDsgY2hhcnNldD11dGYtOCI+Cjx0aXRsZT5MSU1FIGV4cGxhbmF0aW9uPC90aXRsZT4KPHNjcmlwdCBzcmM9Imh0dHBzOi8vY2RuanMuY2xvdWRmbGFyZS5jb20vYWpheC9saWJzL2QzLzMuNS4xNi9kMy5taW4uanMiPjwvc2NyaXB0Pgo8c3R5bGU+CmJvZHkgewogIGZvbnQtZmFtaWx5OiAnTHVjaWRhIEdyYW5kZScsIEx1Y2lkYSBTYW5zIFVuaWNvZGUsIEFyaWFsLCBzYW5zLXNlcmlmOwogIGRpc3BsYXk6IGZsZXg7CiAgZmxleC1kaXJlY3Rpb246IGNvbHVtbjsKICBhbGlnbi1pdGVtczogY2VudGVyOwp9CmsubGVnZW5kIHsKICBiYWNrZ3JvdW5kLWNvbG9yOiB3aGl0ZTsKICBib3JkZXI6IDFweCBzb2xpZCBibGFjazsKICBwYWRkaW5nOiA1cHg7Cn0KLnByZWRpY3Rpb24tcHJvYmFzIHsKICBkaXNwbGF5OiBmbGV4OwogIGp1c3RpZnktY29udGVudDogc3BhY2UtYmV0d2VlbjsKICBwYWRkaW5nOiA1cHggMDsKICBhbGlnbi1pdGVtczogY2VudGVyOwogIG1heC13aWR0aDogNDAwcHg7CiAgZm9udC1zaXplOiAxLjJlbTsKfQoud2VpZ2h0LWJhcnMgewogIGRpc3BsYXk6IGZsZXg7CiAgZmxleC1kaXJlY3Rpb246IGNvbHVtbjsKICBhbGlnbi1pdGVtczogY2VudGVyOwogIHBhZGRpbmc6IDVweDsKICBtYXgtd2lkdGg6IDgwMHB4Owp9Ci53ZWlnaHQtYmFyIHsKICBkaXNwbGF5OiBmbGV4OwogIGp1c3RpZnktY29udGVudDogc3BhY2UtYmV0d2VlbjsKICBhbGlnbi1pdGVtczogY2VudGVyOwogIGJvcmRlcjogMXB4IHNvbGlkIGxpZ2h0Z3JleTsKICBwYWRkaW5nOiA1cHg7CiAgbWFyZ2luLWJvdHRvbTogNXB4OwogIHdpZHRoOiAxMDAlOwogIGJveC1sizing6IGJvcmRlci1ib3g7Cn0KLmZlYXR1cmUgewogIGZsZXgtYmFzaXM6IDQwJTsKICBtYXgtd2lkdGg6IDQwJTsKICB0ZXh0LWFsaWduOiByaWdodDsKfQouaW1wb3J0YW5jZSB7CiAgZmxleC1iYXNpczogNjAlOwogIG1heC13aWR0aDogNjAlOwp9Ci5wb3NpdGl2ZSB7CiAgYmFja2dyb3VuZDogI0MyRTFDNjsKfQouaW1wb3J0YW5jZS5wb3NpdGl2ZSB7CiAgYmFja2dyb3VuZDogIzZCRjg3RDsKfQouaW1wb3J0YW5jZS5uZWdhdGl2ZSB7CiAgYmFja2dyb3VuZDogI0ZGNzA1QzsKfQoubmVnYXRpdmUgewogIGJhY2tncm91bmQ6ICNGN0E0OEM7Cn0KPC9zdHlsZT4KPC9oZWFkPgo8Ym9keT4KPGgyPkxJTUUgVGFidWxhciBFeHBsYW5hdGlvbjwvaDI+CjxkaXYgY2xhc3M9InByZWRpY3Rpb24tcHJvYmFzIj4KICA8ZGl2IGNsYXNzPSJsYWJlbCI+Tm90IENodXJuPC9kaXY+CiAgPGRpdiBjbGFzcz0icHJvYmEiIHN0eWxlPSJ3aWR0aDogNzguODg1NjMwMjA3MDMyNjVweDsgYmFja2dyb3VuZDogI0ZGNzA1QzsiPjAuNzg5PC9kaXY+CjwvZGl2Pgo8ZGl2IGNsYXNzPSJwcmVkaWN0aW9uLXByb2JhcyI+CiAgPGRpdiBjbGFzcz0ibGFiZWwiPkNodXJuPC9kaXY+CiAgPGRpdiBjbGFzcz0icHJvYmEiIHN0eWxlPSJ3aWR0aDogMjEuMTE0MzY5NzkyOTY3MzQ3cHgzOyBiYWNrZ3JvdW5kOiAjNkJGODdEOyI+MC4yMTE8L2Rpdj4KPC9kaXY+CjxkaXYgY2xhc3M9IndlaWdodC1iYXJzIj4KPGgyPkZlYXR1cmUgVmFsdWVzPC9oMj4KICAKICA8ZGl2IGNsYXNzPSJ3ZWlnaHQtYmFyIj4KICAgIDxkaXYgY2xhc3M9ImZlYXR1cmUiPlRlY2hTdXBwb3J0ID0gMTwvZGl2PgogICAgPGRpdiBjbGFzcz0iaW1wb3J0YW5jZSI+CiAgICAgIDxkaXYgY2xhc3M9InBvc2l0aXZlIiBzdHlsZT0iaGVpZ2h0OiAyMHB4OyB3aWR0aDogNDYuODI2MDcyODQ5MjA2MTI1JTsiPjAuMTYyPC9kaXY+CiAgICA8L2Rpdj4KICA8L2Rpdj4KICAKICA8ZGl2IGNsYXNzPSJ3ZWlnaHQtYmFyIj4KICAgIDxkaXYgY2xhc3M9ImZlYXR1cmUiPlBheW1lbnRNZXRob2QgPSAxPC9kaXY+CiAgICA8ZGl2IGNsYXNzPSJpbXBvcnRhbmNlIj4KICAgICAgPGRpdiBjbGFzcz0icG9zaXRpdmUiIHN0eWxlPSJoZWlnaHQ6IDIwcHg7IHdpZHRoOiAzOS45NzU1MTkyNTc0ODIyOCU7Ij4wLjE0MDwvZGl2PgogICAgPC9kaXY+CiAgPC9kaXY+CiAgCiAgPGRpdiBjbGFzcz0id2VpZ2h0LWJhciI+CiAgICA8ZGl2IGNsYXNzPSJmZWF0dXJlIj5TdHJlYW1pbmdNb3ZpZXMgPSAwPC9kaXY+CiAgICA8ZGl2IGNsYXNzPSJpbXBvcnRhbmNlIj4KICAgICAgPGRpdiBjbGFzcz0icG9zaXRpdmUiIHN0eWxlPSJoZWlnaHQ6IDIwcHg7IHdpZHRoOiAyNC4yNjQ0MTYxMjA4Mzk1NCU7Ij4wLjA4NTwvZGl2PgogICAgPC9kaXY+CiAgPC9kaXY+CiAgCiAgPGRpdiBjbGFzcz0id2VpZ2h0LWJhciI+CiAgICA8ZGl2IGNsYXNzPSJmZWF0dXJlIj41OC4wMCAmIzYwOyB0ZW51cmUgJiM2MDsgNzIuMDA8L2Rpdj4KICAgIDxkaXYgY2xhc3M9ImltcG9ydGFuY2UiPgogICAgICA8ZGl2IGNsYXNzPSJuZWdhdGl2ZSIgc3R5bGU9ImhlaWdodDogMjBweDsgd2lkdGg6IDIxLjY2NjgwMjE5NzU2MDc2MyU7Ij4tMC4wNzY8L2Rpdj4KICAgIDwvZGl2PgogIDwvZGl2PgogIAogIDxkaXYgY2xhc3M9IndlaWdodC1iYXIiPgogICAgPGRpdiBjbGFzcz0iZmVhdHVyZSI+MTYgJiM2MDsgQ29udHJhY3Q8L2Rpdj4KICAgIDxkaXYgY2xhc3M9ImltcG9ydGFuY2UiPgogICAgICA8ZGl2IGNsYXNzPSJuZWdhdGl2ZSIgc3R5bE0aGVpZ2h0OiAyMHB4OyB3aWR0aDogNDEuODk1ODc5NzY2NDgxMDklOyI+LTAuMTQ3PC9kaXY+CiAgICA8L2Rpdj4KICA8L2Rpdj4KICAKICA8ZGl2IGNsYXNzPSJ3ZWlnaHQtYmFyIj4KICAgIDxkaXYgY2xhc3M9ImZlYXR1cmUiPjIgJiM2MDsgSW50ZXJuZXRTZXJ2aWNlPC9kaXY+CiAgICA8ZGl2IGNsYXNzPSJpbXBvcnRhbmNlIj4KICAgICAgPGRpdiBjbGFzcz0ibmVnYXRpdmUiIHN0eWxlPSJoZWlnaHQ6IDIwcHg7IHdpZHRoOiA1OS4xNTk4NDg2NDQzNzMxOCU7Ij4tMC4yMDc8L2Rpdj4KICAgIDwvZGl2PgogIDwvZGl2Pgog

### 4. 종합 분석

#### [문제 5] 결과 분석 및 토의 (정답 예시)

**1. Permutation Importance와 SHAP Summary Plot에서 공통적으로 중요하게 나타난 최상위 특성 3가지는 무엇인가요? 두 기법의 결과에 차이가 있다면 그 이유는 무엇일까요?**

-   **공통 최상위 특성**: 양쪽 모두에서 `Contract`(계약 유형), `tenure`(고객 유지 기간), `InternetService`(인터넷 서비스 종류)가 매우 중요한 특성으로 나타났습니다. `MonthlyCharges`(월 요금)와 `TotalCharges`(총 요금)도 상위권에 위치합니다.
-   **차이점과 이유**: 
    -   Permutation Importance는 모델의 최종 성능 지표(여기서는 ROC AUC)가 얼마나 떨어지는지만을 측정하므로 특성의 순수한 '영향력'만을 보여줍니다.
    -   SHAP Summary Plot은 영향력뿐만 아니라, 각 특성 값이 예측에 미치는 **방향성**(이탈 확률을 높이는지, 낮추는지)과 **분포**까지 함께 보여줍니다. 예를 들어, `Contract` 특성의 경우, SHAP plot을 통해 월별 계약(값이 0)이 이탈 확률을 높이고(붉은색 점들이 오른쪽으로), 2년 계약(값이 2)은 이탈 확률을 낮춘다(푸른색 점들이 왼쪽으로)는 구체적인 관계를 알 수 있습니다. 이처럼 SHAP이 더 풍부한 정보를 제공하기 때문에 순위가 약간 다를 수 있습니다.

**2. [문제 3]에서 분석한 '이탈 고객'의 이탈 예측에 가장 큰 영향을 미친 특성은 무엇이었나요? 해당 특성의 값이 어떠했기 때문에 모델이 이탈을 예측했는지 설명하세요.**

-   **가장 큰 영향을 미친 특성**: `Contract` (계약 유형) 입니다.
-   **이유**: SHAP Force Plot에서 `Contract`는 가장 긴 붉은색 막대로 표시되어, 이탈 확률을 가장 크게 높인 요인임을 알 수 있습니다. 이 고객의 `Contract` 값은 0으로, 이는 '월별 계약(Month-to-month)'에 해당합니다. 약정 기간이 짧은 월별 계약 고객은 언제든 쉽게 서비스를 해지할 수 있기 때문에 모델이 이탈 가능성을 매우 높게 판단한 것입니다. 그 외에도 `OnlineSecurity=0`(온라인 보안 서비스 미사용), `TechSupport=0`(기술 지원 미사용) 등도 이탈 예측에 기여했습니다.

**3. [문제 4]에서 분석한 '비이탈 고객'의 비이탈 예측에 가장 큰 영향을 미친 특성은 무엇이었나요?**

-   LIME 결과를 보면 `InternetService > 2`, `Contract > 16`, `tenure > 58`이 비이탈 예측(Not Churn 확률을 낮춤, 즉 Churn 확률을 높임)에 가장 큰 영향을 미쳤습니다. 이는 이 고객이 DSL이나 광랜이 아닌 다른 인터넷 서비스를 사용하고, 장기 계약을 맺었으며, 오랜 기간 고객으로 유지되었음을 의미합니다. 이러한 특성들은 고객 충성도가 높다는 강력한 신호이므로, 모델이 이 고객은 이탈하지 않을 것이라고 강하게 예측한 근거가 됩니다.

**4. 이 모델을 비즈니스에 활용한다면, 어떤 고객에게 어떤 마케팅 활동을 제안할 수 있을까요? 위 분석 결과를 근거로 아이디어를 제시해보세요.**

모델 해석 결과를 바탕으로 다음과 같은 데이터 기반의 마케팅 전략을 수립할 수 있습니다.

-   **타겟 고객군 선정**: `Contract`가 '월별 계약'이고, `tenure`가 짧으며(예: 1년 미만), `OnlineSecurity`나 `TechSupport` 같은 부가 서비스를 이용하지 않는 고객을 '이탈 고위험군'으로 분류합니다.
-   **개인화된 리텐션(고객 유지) 프로그램 제안**:
    -   **장기 계약 유도**: 이탈 고위험군 고객에게 1년 또는 2년 약정으로 전환 시 요금 할인이나 사은품을 제공하는 프로모션을 진행합니다.
    -   **부가 서비스 체험 제공**: `OnlineSecurity`나 `TechSupport` 서비스가 이탈 방지에 긍정적인 영향을 미치므로, 해당 서비스를 몇 개월간 무료로 체험하게 하여 서비스의 가치를 느끼게 하고 유료 전환을 유도합니다.
    -   **요금제 컨설팅**: `MonthlyCharges`가 높은 것도 이탈 요인이므로, 고객의 `TotalCharges`와 사용 패턴을 분석하여 더 저렴하거나 합리적인 요금제를 추천해주는 컨설팅을 제공할 수 있습니다.